In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [45]:
N=1000
epochs=25
mini_batch_size = 100

In [46]:
train_input, train_target, train_classes, \
test_input, test_target, test_classes = \
    prologue.generate_pair_sets(N)

In [47]:
#train_classes = nn.functional.one_hot(train_classes, num_classes=10)
#test_classes = nn.functional.one_hot(test_classes, num_classes=10)

In [48]:
#code from practical 4b
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3),
            nn.MaxPool2d(2, stride=2))
        
        #predict implicitely the class
        self.fc1 = nn.Sequential(
            nn.Linear(32 * 2 * 2, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 10),
            nn.Softmax(1))
        
        #predict the output
        self.fc2 = nn.Sequential(
            nn.Linear(20, 10),
            nn.ReLU(inplace=True),
            nn.Linear(10,2),
            nn.Softmax(1))
        
    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output
    
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        output = self.fc2(torch.cat((output1, output2), 1))
        return output
    
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 25):
    #classification
    criterion = nn.CrossEntropyLoss()
    optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
    eta=1e-1
            
    for e in range(nb_epochs):
        print(e)
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                           train_input.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
            
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            
            optim.zero_grad()
            loss.backward()
            optim.step()
        print(loss)

        
#def compute_nb_errors(model, input_, target_, mini_batch_size):
#    nb_error = 0
#    
#    for b in range(0, input_.size(0), mini_batch_size):
#        output = model(input_.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
#                       input_.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
#        _,pred_class_0 = output[0].max(1)
#        _,pred_class_1 = output[1].max(1)
#
#        for k in range(mini_batch_size):
#            if target_[b+k,0] != pred_class_0[k]:
#                nb_error += 1
#            if target_[b+k,1] != pred_class_1[k]:
#                nb_error += 1
#                
#    return nb_error

def compute_nb_errors(model, input_, target_, mini_batch_size):
    nb_error = 0
    
    for b in range(0, input_.size(0), mini_batch_size):
        output = model(input_.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                       input_.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
        _,pred = output.max(1)

        for k in range(mini_batch_size):
            if target_[b+k] != pred[k]:
                nb_error += 1
                
    return nb_error

# weight sharing -> siamese network 

In [49]:
model = Net(20)

In [50]:
train_model(model, train_input, train_target, mini_batch_size)

0
tensor(0.6917, grad_fn=<NllLossBackward>)
1
tensor(0.6910, grad_fn=<NllLossBackward>)
2
tensor(0.6878, grad_fn=<NllLossBackward>)
3
tensor(0.6872, grad_fn=<NllLossBackward>)
4
tensor(0.6833, grad_fn=<NllLossBackward>)
5
tensor(0.6835, grad_fn=<NllLossBackward>)
6
tensor(0.6799, grad_fn=<NllLossBackward>)
7
tensor(0.6774, grad_fn=<NllLossBackward>)
8
tensor(0.6698, grad_fn=<NllLossBackward>)
9
tensor(0.6598, grad_fn=<NllLossBackward>)
10
tensor(0.6585, grad_fn=<NllLossBackward>)
11
tensor(0.6577, grad_fn=<NllLossBackward>)
12
tensor(0.6461, grad_fn=<NllLossBackward>)
13
tensor(0.6206, grad_fn=<NllLossBackward>)
14
tensor(0.6146, grad_fn=<NllLossBackward>)
15
tensor(0.6134, grad_fn=<NllLossBackward>)
16
tensor(0.5866, grad_fn=<NllLossBackward>)
17
tensor(0.5874, grad_fn=<NllLossBackward>)
18
tensor(0.5700, grad_fn=<NllLossBackward>)
19
tensor(0.5739, grad_fn=<NllLossBackward>)
20
tensor(0.6009, grad_fn=<NllLossBackward>)
21
tensor(0.5605, grad_fn=<NllLossBackward>)
22
tensor(0.5326, gr

In [51]:
nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                  nb_test_errors, test_input.size(0)))

test error Net 23.80% 238/1000
